<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeAutomatizado/blob/master/notebooks/1c_clasificador_bayesiano_categorico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador bayesiano ingenuo categórico
En este libreta revisaremos dos formas de programar el clasificador bayesiano ingenuo. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB

## Conjunto de datos
Usaremos un conjunto de datos de partidas del juego Go, la cual contiene dos atributos categóricos: ID del jugador con fichas negras y ID del jugador con fichas blancas. Como respuesta tenemos el ganador de la partida.

In [2]:
df_ent = pd.read_csv('https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/data/partidas_entrenamiento.csv')
df_prueba = pd.read_csv('https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/data/partidas_prueba.csv')

Visualizamos los primeros registros.

In [3]:
df_ent.head(5)

,id_blancas,id_negras,ganador
0,3,4,1
1,7,8,1
2,40,10,1
3,16,17,1
4,8,18,1


In [4]:
df_prueba.head(5)

,id_blancas,id_negras,ganador
0,35,43,1
1,45,9,0
2,55,106,0
3,35,43,1
4,35,43,1


Examinamos algunos estadísticos.

In [5]:
df_prueba.astype('object').describe()

,id_blancas,id_negras,ganador
count,115,115,115
unique,34,31,2
top,35,43,1
freq,23,47,82


## Programación con NumPy
Definimos una función para obtener la probabilidad de una categoría.


In [6]:
def categorical(c, q):
    """
    Distribución categórica
    """
    return q[int(c)]

Definimos nuestro clasificador bayesiano ingenuo para la tarea de predicción del ganador de una partida dados los jugadores y con qué fichas juega. 

In [7]:
class CategoricalNB:
  "Clasificador bayesiano ingenuo binario con características categóricas"

  def prob_clase_(self):
    """
    Calcula probabilidad de la clase
    """
    return self.qc

  def prob_cond_clase_(self, X):
    """
    Calcula probabilidad de atributo dada la clase
    """
    v = np.zeros((X.shape[0], self.clases.size))
    for k in range(X.shape[0]):
      for i in range(self.clases.size):
        for j in range(self.n_atrib):
          v[k, i] = v[k, i] + categorical(X[k, j], self.qa[i, j, :])

    return v

  def fit(self, X, y, alfa = 2):
    """
    Entrena clasificador bayesiano ingenuo
    """
    # Calcula parámetros de distribución a priori (categórica)
    n = X.shape[0]
    self.clases = np.unique(y)
    self.n_clases = self.clases.size
    self.qc = np.zeros(self.clases.size)
    for i,c in enumerate(self.clases):
      # Misma alfa para todas las categorías
      self.qc[i] = (np.sum(y == c) + alfa - 1) / (n + alfa * self.n_clases - self.n_clases)

    # Escala logarítmica para parámetros de a priori
    self.qc[self.qc == 0] = np.nextafter(0, 1)
    self.qc[self.qc == 1] = np.nextafter(1, 0)
    self.qc = np.log(self.qc)

    # Calcula parámetros de verosimilitud (categórica)
    self.n_atrib = X.shape[1]

    # Presupone mismas categorías en los dos atributos (mismos posibles jugadores)
    self.n_cat = int(np.max(X)) + 1
    self.qa = np.zeros((self.n_clases, self.n_atrib, self.n_cat))

    # Estima parámetros para cada clase, atributo y categoría (máximo a posteriori)
    for c in self.clases:
      for i in range(self.n_atrib):
        X_ci = X[np.where(y == c), i]
        for j in range(self.n_cat): 
          # Misma alfa para todas las categorías
          self.qa[c, i, j] = (np.sum(X_ci == j) + alfa - 1) / (X_ci.shape[0] + alfa * self.n_cat - self.n_cat)

    # Usa escala logarítmica para parámetros de verosimilitud
    self.qa[self.qa == 0] = np.nextafter(0, 1)
    self.qa[self.qa == 1] = np.nextafter(1, 0)
    self.qa = np.log(self.qa)

  def predict(self, X):
    """
    Predice clases dada un conjunto de datos
    """
    aposteriori = self.prob_clase_() + self.prob_cond_clase_(X)
    return np.argmax(aposteriori, axis = 1), np.max(aposteriori, axis = 1)

  def score(self, X, y):
    """
    Calcula exactitud dado datos
    """
    preds, probs = self.predict(X)
    return np.mean(preds == y) * 100

Entrenamos y evaluamos el clasificador. 

In [8]:
X_ent = df_ent[['id_blancas', 'id_negras']].to_numpy()
y_ent = df_ent['ganador'].to_numpy()

X_prueba = df_prueba[['id_blancas', 'id_negras']].to_numpy()
y_prueba = df_prueba['ganador'].to_numpy()

cnb = CategoricalNB()
cnb.fit(X_ent, y_ent)

print('Exactitud en entrenamiento: {0:.2f}%'.format(cnb.score(X_ent, y_ent)))
print('Exactitud en prueba: {0:.2f}%'.format(cnb.score(X_prueba, y_prueba)))

Exactitud en entrenamiento: 84.64%
Exactitud en prueba: 72.17%



## Programación con scikit-learn
Convertimos cada atributo categórico a 1 de K.

In [9]:
n_values = int(np.max(X_ent[:,:2])) + 1
categorias = [c for c in range(n_values)]

X0_ent = X_ent[:, 0]
ohe0 = OneHotEncoder(categories=[categorias])
X0_ent_ohe = ohe0.fit_transform(X0_ent[:, np.newaxis])

X1_ent = X_ent[:, 1]
ohe1 = OneHotEncoder(categories=[categorias])
X1_ent_ohe = ohe1.fit_transform(X1_ent[:, np.newaxis])

Generamos modelos de clasificación bayesiana multinomial por separado por cada atributo.

In [10]:
mnb0 = MultinomialNB()
mnb0.fit(X0_ent_ohe, y_ent)
mnb1 = MultinomialNB()
mnb1 = mnb1.fit(X1_ent_ohe, y_ent)

Calcula probabilidad de clases dados datos de entrenamiento por cada modelo.

In [11]:
pe0 = mnb0.predict_log_proba(X0_ent_ohe)
pe1 = mnb1.predict_log_proba(X1_ent_ohe)

Sumamos probabilidades y restamos a priori extra (se sumó 2 veces porque fueron 2 modelos).

In [12]:
pet = pe0 + pe1 - mnb0.class_log_prior_ 
pred_ent = pet.argmax(axis = 1)

Convertimos cada atributo de los datos de prueba a 1 de K.

In [13]:
X0_prueba = X_prueba[:, 0]
X1_prueba = X_prueba[:, 1]

X0_prueba_ohe = ohe0.transform(X0_prueba[:, np.newaxis])
X1_prueba_ohe = ohe1.transform(X1_prueba[:, np.newaxis])

Calculamos probabilidad de clases dados datos de entrenamiento por cada modelo.

In [14]:
pv0 = mnb0.predict_log_proba(X0_prueba_ohe)
pv1 = mnb1.predict_log_proba(X1_prueba_ohe)

Sumamos probabilidades y restamos a priori extra (se sumó 2 veces porque fueron 2 modelos).

In [15]:
pvt = pv0 + pv1 - mnb0.class_log_prior_
pred_prueba = pvt.argmax(axis = 1)

Calculamos la exaxctitud para los conjuntos de entrenamiento y prueba.

In [16]:
print('Exactitud en entrenamiento: {0:.2f}%'.format(np.mean(pred_ent == y_ent) * 100))
print('Exactitud en prueba: {0:.2f}%'.format(np.mean(pred_prueba == y_prueba) * 100))

Exactitud en entrenamiento: 84.64%
Exactitud en prueba: 69.57%
